In [1]:
import numpy as np
import pandas as pd


In [2]:
df=pd.read_csv('/content/spotify_millsongdata.csv')


In [3]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [4]:
df=df.sample(5000).drop("link", axis=1).reset_index(drop=True)
df.tail(5)


,artist,song,text
4995,W.A.S.P.,B.A.D.,"You hear the cries of love, a sad tune \r\nAn..."
4996,Offspring,Huck It,Hitting walls for a feet \r\nWith our friends...
4997,The Monkees,Never Enough,From Justus \r\nWhy is it never enough \r\nN...
4998,Neil Diamond,Fool That I Am,"Fool that I am \r\nOh, I let love past me on ..."
4999,Jimmy Buffett,Desdemonas Building A Rocket Ship,"Desdemona's building a rocket ship, Desdemona'..."


In [5]:
df.head(10)

,artist,song,text
0,The Monkees,Mommy And Daddy,"Ask your mommy and daddy, ""What happened to th..."
1,Dolly Parton,Dump The Dude,"Hey girl, I wanna give you some friendly advic..."
2,Nine Inch Nails,Head Down,You \r\nWhat you looking at? \r\nHead down ...
3,Judy Garland,Hoe Down,When our Grandmas and our Grandpas where just ...
4,Emmylou Harris,Even Cowgirls Get The Blues,(Rodney Crowell) \r\nShe's a rounder I can te...
5,King Diamond,Sarah's Night,"Monday night, Tuesday morning, master bedroom ..."
6,Alice Cooper,Little By Little,Black leather gloves and your lipstick shines ...
7,Qntal,Unter Der Linden,"Under der linden an der heide, \r\nD unser zw..."
8,Omd,Stay,You say you gotta go now \r\nWell that's ok ...
9,Kenny Loggins,Conviction Of The Heart,Where are the dreams that we once had? \r\nTh...


In [6]:
df.shape

(5000, 3)

In [7]:
df['text'][0]

'Ask your mommy and daddy, "What happened to the Indian?  \r\nHow come they\'re all living in places  \r\nWith too much snow or too much sand?"  \r\nTell your mommy, "I got a funny feeling deep inside of me"  \r\nAsk your daddy, "Is it really as bad as people make it seem?"  \r\n  \r\nWa ta tick it ta too too  \r\nAsk your mommy why everybody swallows all those little pills  \r\nAsk your daddy why that soldier doesn\'t care who he kills  \r\nAfter they\'ve put you to sleep and tucked you safely down in your bed  \r\nWhisper, "Mommy and Daddy would you rather that  \r\nI learned it from my friends, instead?  \r\nDo you think I\'m to young to know, to see, to feel, or hear  \r\nMy questions need an answer or a vacuum will appear"  \r\n  \r\nDon\'t be surprised if they turn and walk away  \r\nAnd tell your mommy and daddy that you love them anyway  \r\nThat you love them anyway  \r\nGive them a kiss  \r\nLove will set you free  \r\nKings of EMI\r\n\r\n'

In [8]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex=True)

In [11]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer

ps= PorterStemmer()

def token(txt):
     token= nltk.word_tokenize(txt)
     a =[ps.stem(w) for w in token]
     return " ".join(a)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
df['text'].apply(lambda x: token(x))

0       ask your mommi and daddi , `` what happen to t...
1       hey girl , i wan na give you some friendli adv...
2       you what you look at ? head down too late for ...
3       when our grandma and our grandpa where just gi...
4       ( rodney crowel ) she 's a rounder i can tell ...
                              ...                        
4995    you hear the cri of love , a sad tune and feel...
4996    hit wall for a feet with our friend and we hea...
4997    from justu whi is it never enough no matter wh...
4998    fool that i am oh , i let love past me on by o...
4999    desdemona 's build a rocket ship , desdemona '...
Name: text, Length: 5000, dtype: object

In [13]:
token("you are caring cared care")

'you are care care care'

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel


In [15]:
tfid= TfidfVectorizer(analyzer='word', stop_words='english')

In [16]:
matrix= tfid.fit_transform(df['text'])

In [18]:
cos_sim=  linear_kernel(matrix, matrix)

In [19]:
indices= pd.Series(df.index, index=df['song'])
indices

song
Mommy And Daddy                         0
Dump The Dude                           1
Head Down                               2
Hoe Down                                3
Even Cowgirls Get The Blues             4
                                     ... 
B.A.D.                               4995
Huck It                              4996
Never Enough                         4997
Fool That I Am                       4998
Desdemonas Building A Rocket Ship    4999
Length: 5000, dtype: int64

In [20]:
indices['Just Good Friends']

3566

In [21]:
df[df['song']=='Prove My Love'].index[0]

370

In [ ]:

def recommender(title, cos_sim=cos_sim):
       idx=print(indices[title])
       sim_scores= sorted((list(enumerate(cos_sim[idx]))), key=lambda x: x[1], reverse=True)
       sim_scores=sorted(sim_scores, key=lambda x: x[1], reverse=True)
       for i in sim_scores:
             print(i)

       sim_scores=sim_scores[1:5]
       print(sim_scores)
       for i in sim_scores:
                 print[df['song'].iloc[i]]



recommender('I Froze Up')





1
[]


In [35]:
def get_recommendations(title, cos_sim=cos_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))

    # Sort the songs based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar songs
    sim_scores = sim_scores[1:21]
    sim_scores.sort()
    # Get the song indices
    song_index = [i[0] for i in sim_scores]
    #song_index = song_index
    print(df['song'].iloc[song_index])

    # Return the top 10 most similar songs
    #return song_index[1:5]


get_recommendations("A World Without Hereos")



134        Living In A World Without Her
363                     This Glass World
402                      Mr. Know It All
711                                 Voix
1281                      I'm Against It
1367                   You Don't Know Me
1641                     A Dream Of Siam
2096                    World Down Under
2249                         Do You Know
2288                         Dream World
2328                     I Am Two People
2430              There's A Place For Us
2562                       The One Thing
2912                    I Don't Know Why
3412          If I Had The World To Give
3479              Can't Change The World
3567                    But I Dropped It
3940    I Don't Want To Change The World
4101                        In His World
4850          How You See The World No.2
Name: song, dtype: object


In [37]:
import pickle
pickle.dump(cos_sim,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))